# Lọc Cộng Tác (Collaborative Filtering)

Thời lượng ước tính: **45** phút

## Mục tiêu

Sau khi hoàn thành lab, này bạn sẽ có thể:

-   Tạo hệ thống đề xuất (recommendation system) dựa trên lọc cộng tác


Hệ thống đề xuất là một tập hợp các thuật toán được sử dụng để đề xuất các mặt hàng cho người dùng dựa trên thông tin được lấy từ người dùng. Những hệ thống này đã trở nên phổ biến và có thể thấy nhiều ở các cửa hàng trực tuyến, cơ sở dữ liệu phim ảnh và công cụ tìm việc. Trong notebook này, chúng ta sẽ khám phá các hệ thống đề xuất dựa trên Lọc cộng tác (Collaborative Filtering) và triển khai phiên bản đơn giản của một hệ thống sử dụng Python và thư viện Pandas.


<hr>

<a id="ref2"></a>

# Tiền xử lý (Preprocessing)


Trước tiên, chúng ta hãy thực hiện import:


In [ ]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Bây giờ hãy đọc từng tệp vào các khung dữ liệu (Dataframe):


In [ ]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('movies.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('ratings.csv')

Hãy cũng xem một chút cách tổ chức của mỗi khung dữ liệu trong số đó:


In [ ]:
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
movies_df.head()

Vậy, mỗi bộ phim có một ID duy nhất, một tiêu đề với năm phát hành (Có thể chứa các ký tự unicode) và một số thể loại khác nhau trong cùng một lĩnh vực. Hãy xóa year khỏi cột title và đặt nó đó vào cột riêng bằng hàm [extract](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.extract.html#pandas.Series.str.extract?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork-20718538&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork-20718538&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ) của Pandas.


Hãy xóa year khỏi cột **title** bằng cách sử dụng hàm thay thế của pandas và lưu trữ trong cột **year** mới.

In [ ]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

Hãy xem kết quả!


In [ ]:
movies_df.head()

Cùng với đó, hãy bỏ cột genres vì chúng ta không cần nó cho hệ thống đề xuất này.


In [ ]:
#Dropping the genres column
movies_df = movies_df.drop('genres', 1)

Đây là khung dữ liệu final movies:


In [ ]:
movies_df.head()

Tiếp theo, hãy xem khung dữ liệu ratings.


In [ ]:
ratings_df.head()

Mỗi hàng trong khung dữ liệu ratings có một id người dùng được liên kết với ít nhất một phim, xếp hạng và dấu thời gian hiển thị khi họ đánh giá phim đó. Chúng ta sẽ không cần cột dấu thời gian, vì vậy hãy bỏ để tiết kiệm bộ nhớ.


In [ ]:
#Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', 1)

Dưới đây là khung dữ liệu ratings:


In [ ]:
ratings_df.head()

<hr>

<a id="ref3"></a>

# Lọc cộng tác


Giờ đã đến lúc bắt đầu công việc với hệ thống đề xuất.

Kỹ thuật đầu tiên chúng ta sẽ xem xét được gọi là **Lọc cộng tác (Collaborative Filtering)**, còn được gọi là **User-User Filtering**. Như tên thay thế của nó gợi ý, kỹ thuật này sử dụng các user (người dùng) khác để đề xuất các mặt hàng cho input user. Nó cố gắng tìm các người dùng có sở thích và quan điểm tương tự làm đầu vào và sau đó đề xuất các mặt hàng mà họ thích cho đầu vào. Có một số phương pháp tìm kiếm người dùng tương tự (Thậm chí một số phương pháp sử dụng Machine Learning) và phương pháp chúng ta sẽ sử dụng ở đây dựa trên **Hàm tương quan Pearson**.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/images/User_Item.png" width=800px>

Quy trình tạo hệ thống đề xuất Dựa trên Người dùng như sau:

- Chọn một người dùng với các bộ phim mà người dùng đã xem
- Dựa trên xếp hạng phim của người ấy, hãy tìm top X neighbor (các bộ phim tương tự)
- Nhận bản ghi phim đã xem của người dùng cho từng neighbor.
- Tính hệ số tương đồng bằng cách sử dụng một số công thức
- Đề xuất các mặt hàng có hệ số cao nhất

Hãy bắt đầu bằng cách tạo đầu vào người dùng (user input) để đề xuất phim:

Lưu ý: Để thêm nhiều phim hơn, chỉ cần tăng số lượng phần tử trong userInput. Vui lòng thêm nhiều hơn vào! Chỉ cần đảm bảo viết hoa chữu cái đầu và nếu một bộ phim bắt đầu bằng "The", chẳng hạn như "The Matrix" thì hãy viết nó như sau: 'Matrix, The'.


In [ ]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

#### Thêm movieId vào input user

Để hoàn tất đầu vào (input), hãy trích xuất ID của đầu vào phim từ khung dữ liệu movies và thêm chúng vào đó.

Chúng ta có thể đạt được điều này trước tiên bằng cách lọc ra các hàng chứa tiêu đề của đầu vào phim và sau đó hợp nhất tập con này với khung dữ liệu đầu vào. Chúng ta cũng loại bỏ các cột không cần thiết cho đầu vào để tiết kiệm dung lượng bộ nhớ.


In [ ]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

#### Những người dùng đã xem những bộ phim tương tự

Bây giờ với ID phim trong in, chúng ta có thể lấy tập con (subset) người dùng đã xem và đánh giá phim trong input.


In [ ]:
#Filtering out users that have watched movies that the input has watched and storing it
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

Bây giờ hãy nhóm các hàng theo ID người dùng.


In [ ]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userId'])

hãy xem xét một trong những người dùng, ví dụ: userID=1130

In [ ]:
userSubsetGroup.get_group(1130)

Cũng hãy sắp xếp các nhóm này để những người dùng có chung nhiều phim nhất với đầu vào có mức độ ưu tiên cao hơn. Điều này cung cấp một đề xuất phong phú hơn vì chúng ta sẽ không xem xét từng người dùng.

In [ ]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

Bây giờ chúng ta hãy xem xét người dùng đầu tiên


In [ ]:
userSubsetGroup[0:3]

#### Sự giống nhau giữa user với input user

Tiếp theo, chúng ta sẽ so sánh tất cả người dùng (không thực sự là tất cả !!!) với người dùng được chỉ định và tìm ra người tương tự nhất.
Chúng ta sẽ tìm hiểu mức độ tương tự của mỗi người dùng với đầu vào thông qua **Hệ số tương quan Pearson**. Nó được sử dụng để đo độ mạnh của một liên kết tuyến tính giữa hai biến. Công thức để tìm hệ số này giữa các tập X và Y với N giá trị có thể thấy trong hình dưới đây.

Tại sao là Tương quan Pearson (Pearson Correlation)?

Tương quan Pearson là bất biến đối với tỷ lệ, tức là nhân tất cả các phần tử với một hằng số khác 0 hoặc thêm bất kỳ hằng số nào vào tất cả các phần tử. Ví dụ: nếu bạn có 2 vectơ X và Y, thì pearson(X,Y) == pearson(X, 2 * Y + 3). Đây là một thuộc tính khá quan trọng trong hệ thống đề xuất vì chẳng hạn hai người dùng có thể xếp hạng hai loạt mặt hàng hoàn toàn khác nhau về tỷ lệ tuyệt đối, nhưng họ sẽ là những người dùng tương tự (tức là có ý tưởng tương tự) với tỷ lệ tương tự ở các thang điểm khác nhau.

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

Các giá trị được đưa ra bởi công thức thay đổi từ r = -1 đến r = 1, trong đó 1 tạo thành mối tương quan trực tiếp giữa hai thực thể (có nghĩa là tương quan dương hoàn hảo) và -1 tạo thành tương quan âm hoàn hảo.

Trong trường hợp của chúng ta, 1 có nghĩa là hai người dùng có sở thích giống nhau trong khi -1 có nghĩa là ngược lại.


Chúng ta sẽ chọn một nhóm nhỏ người dùng để lặp lại. Giới hạn này được áp dụng bởi vì chúng ta không muốn lãng phí quá nhiều thời gian để duyệt từng người dùng.

In [ ]:
userSubsetGroup = userSubsetGroup[0:100]

Bây giờ, hãy tính toán Tương quan Pearson giữa input user và nhóm tập hợp con, và lưu trữ nó trong dictionary, trong đó khóa là Id người dùng và giá trị là hệ số

In [ ]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


In [ ]:
pearsonCorrelationDict.items()

In [ ]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

#### Top x user tương tự với input user

Bây giờ, hãy lấy top 50 user giống với đầu vào nhất.


In [ ]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

Bây giờ, hãy bắt đầu đề xuất phim cho input user.

#### Xếp hạng của những người dùng đã chọn cho tất cả các phim

Chúng ta sẽ làm điều này bằng cách lấy trung bình có trọng số của xếp hạng các bộ phim sử dụng Tương quan Pearson làm trọng số. Nhưng để làm được điều này, trước tiên chúng ta cần lấy những bộ phim mà người dùng đã xem trong **pearsonDF** từ khung dữ liệu ratings và sau đó lưu trữ mối tương quan của chúng trong một cột mới _similarityIndex ". Ta thực hiện bằng cách hợp nhất hai bảng này .


In [ ]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

Bây giờ, tất cả những gì chúng ta cần làm chỉ đơn giản là nhân xếp hạng phim với trọng số của nó (Chỉ số tương tự), sau đó cộng các xếp hạng mới và chia cho tổng trọng số.

Chúng ta có thể dễ dàng thực hiện việc này bằng cách nhân hai cột, rồi nhóm khung dữ liệu lại theo movieId và sau đó chia hai cột:

Nó cho thấy ý tưởng của tất cả các người dùng tương tự về phim ứng cử cho input user:

In [ ]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

In [ ]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

In [ ]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

Bây giờ chúng ta hãy sắp xếp nó và xem 20 bộ phim hàng đầu mà thuật toán đề xuất!

In [ ]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

In [ ]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

### Ưu điểm và Nhược điểm của Lọc cộng tác

##### Ưu điểm

-   Xem xét xếp hạng của người dùng khác
-   Không cần nghiên cứu hoặc trích xuất thông tin từ các mục được đề xuất
-   Thích ứng với sở thích của người dùng có thể thay đổi theo thời gian

##### Nhược điểm

-   Hàm tiệm cận có thể chậm
-   Có thể chỉ ước tính được lượng người dùng nhỏ
-   Các vấn đề về quyền riêng tư khi cố gắng tìm hiểu sở thích của người dùng


### Cảm ơn bạn đã hoàn thành lab này!

Nguồn bài Lab: **IBM**